# Get Recommendations for Anime

Сам по себе сайт дурацкий, так как часто падает. Видимо, это защита от Web Scraping, чем мы сейчас и занимаемся.

Начнем с самого легкого, так как дальше нужно будет собирать целый набор данных. Удачи.

Вот документация: https://jikan.docs.apiary.io/#reference

А вот оболочка для API: https://github.com/abhinavk99/jikanpy

In [10]:
import jikanpy as jp
import pandas as pd
import aiohttp
from time import sleep, time

In [7]:
anime_dataset = pd.read_csv('./datasets/anime_dataset/anime.csv')

In [8]:
ids = anime_dataset.MAL_ID.tolist()

In [14]:
jikan = jp.Jikan()

In [15]:
def get_cleared_recs(uncleared_recs):
    titles = []
    mal_ids = []
    i = 0
    while i != 7:
        try:
            title = uncleared_recs['recommendations'][i]['title']
            mal_id = uncleared_recs['recommendations'][i]['mal_id']
            titles.append(title)
            mal_ids.append(mal_id)
            i += 1
        except IndexError:
            titles.append('No')
            mal_ids.append('No')
            i += 1
    return titles, mal_ids

In [16]:
def get_recs(id):
    uncleared_recs = jikan.anime(id, extension='recommendations')
    recs = get_cleared_recs(uncleared_recs)
    return recs

In [24]:
anime_recs = pd.DataFrame(columns=['ID', 'Reco_Title', 'Reco_ID'])

Из-за того, что, скорее всего, за один заход не получится получить все значения, пришлось сделать так, чтобы если значение уже есть в DataFrame, оно пропускалось, если нет, то исполняется обычный код.

In [25]:
start_time = time()
i = 0
not_responsed_ids = []
for item_id in ids:
    try:
        if item_id not in anime_recs.ID.tolist():
            titles, mal_ids = get_recs(item_id)
            for i in range(7):
                anime_recs.loc[len(anime_recs.index)] = [item_id, titles[i], mal_ids[i]]
            if len(anime_recs) % 100 == 0:
                print(len(anime_recs))
            sleep(2)
        if item_id in anime_recs.ID.tolist():
            continue
    except jp.APIException:
        not_responsed_ids.append(item_id)
        sleep(60)
print("--- %s seconds ---" % (time() - start_time))

700
--- 217.74237871170044 seconds ---


In [13]:
async def get_recs(ids):
    async with aiohttp.ClientSession() as session:
        all_recs = pd.DataFrame(columns=['ID', 'Reco_Title', 'Reco_ID'])
        for i in ids:
            async with session.get('https://api.jikan.moe/v3/anime/' + str(i) + '/recommendations/') as resp:
                anime = await resp.json()
                # на тот случай, если аниме под id равному i несуществует
                if 'recommendations' in anime.keys():
                    # на тот случай, если у нас меньше чем 7 рекомендаций
                    for j in range(len(anime['recommendations'][:7])):
                        all_recs.loc[len(all_recs.index)] = [i, anime['recommendations'][j]['title'], anime['recommendations'][j]['mal_id']]
        return all_recs

In [11]:
start_time = time()
anime_recs = await get_recs(ids[:100])
print('--- %s seconds ---' % (time() - start_time))

--- 666.3386619091034 seconds ---


In [ ]:
anime_recs.to_csv('C:/Users/ASDW/Python/Projects/Recosys 2.0/datasets/anime_dataset/recommendations.csv')

In [26]:
anime_recs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 700 entries, 0 to 699
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID          700 non-null    object
 1   Reco_Title  700 non-null    object
 2   Reco_ID     700 non-null    object
dtypes: object(3)
memory usage: 21.9+ KB


In [27]:
217*170

36890

In [28]:
36890/3600

10.247222222222222